In [ ]:
# Baixa os arquivos zipados do Google Drive contendo os dados
!gdown 169RiUEtrp4cD1zN-AjEWvSUyGOAcDgXb
!gdown 1Be4s_PFa2Uyc67iOqAEyMc_Ce5WiJ_fH
!gdown 1fc6DWa9hnxYtRhuHntdXllPrrVQ54NiH

Downloading...
From (original): https://drive.google.com/uc?id=169RiUEtrp4cD1zN-AjEWvSUyGOAcDgXb
From (redirected): https://drive.google.com/uc?id=169RiUEtrp4cD1zN-AjEWvSUyGOAcDgXb&confirm=t&uuid=fd8eed84-9b56-4f9f-b289-67dd76bfaa9f
To: c:\Users\luqui\OneDrive\lvcas\USP\7o Período\Redes Neurais\trabalho\PianoSkillsAssessment\processed_data.zip

  0%|          | 0.00/327M [00:00<?, ?B/s]
  0%|          | 524k/327M [00:00<02:29, 2.19MB/s]
  1%|          | 2.62M/327M [00:00<00:37, 8.64MB/s]
  3%|▎         | 8.39M/327M [00:00<00:13, 24.3MB/s]
  4%|▍         | 14.7M/327M [00:00<00:08, 35.6MB/s]
  6%|▋         | 21.0M/327M [00:00<00:07, 42.8MB/s]
  8%|▊         | 27.3M/327M [00:00<00:06, 48.6MB/s]
 11%|█         | 36.7M/327M [00:00<00:04, 61.3MB/s]
 14%|█▎        | 44.6M/327M [00:00<00:04, 65.9MB/s]
 16%|█▌        | 52.4M/327M [00:01<00:04, 67.5MB/s]
 18%|█▊        | 60.3M/327M [00:01<00:03, 70.1MB/s]
 21%|██        | 68.2M/327M [00:01<00:03, 72.1MB/s]
 23%|██▎       | 75.5M/327M [00:01<00:0

In [ ]:
# Descompacta os arquivos zip no ambiente
!tar -xf processed_data.zip
!tar -xf audio_samples.zip
!tar -xf checkpoints_batch4.zip

In [ ]:
# Importa todas as bibliotecas e módulos Python necessários para construir o modelo, carregar os dados, treinar e avaliar.
import os
import torch
from torch import nn
from preprocessed_dataset import PreprocessedDataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from dataloader_multimodal import VideoDataset
from opts import *
from collections import Counter

In [ ]:
# Define os caminhos para os diretórios onde os dados de treino e teste pré-processados estão localizados.
PROCESSED_TRAIN_DIR = './processed_data/train/'
PROCESSED_TEST_DIR = './processed_data/test/'

In [ ]:
# Esta classe define a arquitetura da rede neural, baseada em uma ResNet18 pré-treinada,
# adaptada para realizar tanto a classificação do nível de habilidade quanto a regressão.
class AuralSkillClassifier(nn.Module):
    # Construtor da classe, responsável por criar uma instância sua, herdar os atributos
    # e métodos da classe pai (nn.Module) e definir o restante dos atributos da classe
    def __init__(self, num_classes=10):
        super().__init__()

        # inicia um backbone da rede com a ResNet18 pré-treinada
        self.backbone = resnet18(weights=ResNet18_Weights.DEFAULT)

        # modifica a primeira camada para receber apenas 1 canal de entrada
        # # (pois o espectrograma está em escala de cinza)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

        # remove a camada de classificação original da ResNet, substituindo-a por uma nn.Identity()
        self.backbone.fc = nn.Identity()

        # cria uma nova camada de classificação, conforme descrita no artigo, com 512 valores de entrada e 128 de saída,
        # posteriormente passando por uma camada que faz a classificação de fato, com 10 valores de saída (correspondentes às classes)
        self.classifier = nn.Sequential(
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )
        self.classification_head = nn.Linear(128, num_classes)
        self.regression_head = nn.Linear(128, 1)
        
    # Define o fluxo de dados através da rede neural.
    # 'x' representa o tensor de entrada (espectrograma de áudio).
    def forward(self, x):
        features = self.backbone(x) # passa a entrada pela rede ResNet
        features_128 = self.classifier(features) # passa o resultado da ResNet pela camada final de classificação
        logits_cls = self.classification_head(features_128)
        output_reg = self.regression_head(features_128)

        return logits_cls, output_reg

In [32]:
def calculate_class_weights(dataset, num_classes=10):
    """
    Calcula os pesos para cada classe com base na frequência inversa das amostras.
    Esta função percorre o dataset uma vez para contar as ocorrências de cada classe
    e depois calcula os pesos.

    :param dataset: Uma instância do seu objeto de Dataset (ex: PreprocessedDataset).
    :param num_classes: O número total de classes no seu problema.
    :return: Um tensor do PyTorch de formato [num_classes] com o peso para cada classe.
    """
    print("Iniciando o cálculo dos pesos das classes...")

    # --- PASSO 1: Contar as Amostras de Cada Classe ---
    
    # A forma mais eficiente de contar é usar um DataLoader para iterar sobre os dados.
    # batch_size pode ser maior para acelerar a contagem. shuffle=False não é necessário aqui.
    loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4)
    
    # Usaremos um Counter para armazenar as contagens de cada label.
    class_counts = Counter()
    
    # Itera sobre o dataset para contar os labels
    for batch_data in loader:
        labels = batch_data['player_lvl']
        # .tolist() converte o tensor de labels do lote para uma lista Python
        class_counts.update(labels.tolist())

    # Transforma o Counter em uma lista ordenada pelo índice da classe (de 0 a 9)
    # Se uma classe não aparecer, sua contagem será 0.
    counts = [class_counts.get(i, 0) for i in range(num_classes)]
    print(f"Contagem de amostras por classe: {counts}")
    

    # --- PASSO 2: Calcular os Pesos ---
    
    # A fórmula é o inverso da frequência: peso = 1 / contagem
    # Usamos uma lista para guardar os pesos calculados.
    weights = []
    for count in counts:
        # Lida com o caso de uma classe não ter amostras para evitar divisão por zero
        if count == 0:
            weights.append(0.0)
        else:
            weights.append(1.0 / count)

    # Converte a lista de pesos em um tensor do PyTorch do tipo float
    weights_tensor = torch.tensor(weights, dtype=torch.float)
    
    # Opcional, mas recomendado: Normalizar os pesos para que a soma deles não seja muito grande,
    # o que poderia desestabilizar o treinamento. Aqui, normalizamos pela soma.
    weights_tensor = weights_tensor / weights_tensor.sum()
    
    print(f"Pesos calculados para as classes: {weights_tensor}")
    print("Cálculo de pesos concluído.")
    
    return weights_tensor


In [ ]:
# Define qual dispositivo será utilizado para o processamento dos cálculos da rede neural:
# GPU (CUDA) se ela estiver disponível, caso contrário, a CPU.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando o dispositivo: {device}")

# Cria uma instância do modelo e a move para o dispositivo
model = AuralSkillClassifier(num_classes=10).to(device)

Usando o dispositivo: cuda


In [ ]:
# Inicializa o dataset de treino, calcula os pesos das classes para balanceamento,
# configura o DataLoader para carregamento em lotes e define o otimizador Adam.
train_dataset = PreprocessedDataset(data_dir=PROCESSED_TRAIN_DIR)
pesos_tensor = calculate_class_weights(train_dataset).to(device)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

Dataset encontrado. Número de amostras: 516
Iniciando o cálculo dos pesos das classes...
Contagem de amostras por classe: [18, 19, 34, 16, 27, 22, 72, 89, 157, 62]
Pesos calculados para as classes: tensor([0.1682, 0.1594, 0.0891, 0.1893, 0.1122, 0.1377, 0.0421, 0.0340, 0.0193,
        0.0488])
Cálculo de pesos concluído.


In [ ]:
# Define as funções de perda que serão utilizadas para calcular os erros do modelo durante o treinamento.
criterion_cls = nn.CrossEntropyLoss(weight=pesos_tensor)
criterion_reg_l1 = nn.L1Loss()
criterion_reg_l2 = nn.MSELoss()

In [48]:
# Caminho para o modelo pré-treinado
checkpoint_dir = './checkpoints_batch4/'
checkpoint_path = './checkpoints_batch4/model_epoch_100.pt'

# Tentar carregar o modelo e o otimizador a partir do checkpoint
if os.path.exists(checkpoint_path):
    print(f"Checkpoint encontrado. Carregando o modelo pré-treinado de {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    loss = checkpoint['loss']  # Perda final do último lote da época
    print(f"Modelo carregado com sucesso a partir da época {start_epoch}. Continuando o treinamento...")
else:
    print("Nenhum checkpoint encontrado. Iniciando treinamento do zero.")
    start_epoch = 0  # Começar do início

# --- INÍCIO DO TREINAMENTO ---

# Coloque o modelo no modo de treinamento
model.train()

# Número de épocas de treinamento
num_epochs = 100

for epoch in range(start_epoch, num_epochs):
    print(f"--- Iniciando Época {epoch+1}/{num_epochs} ---")

    # Para cada lote de dados
    for i, batch_data in enumerate(train_loader):
        print(f"  Processando lote {i+1}/{len(train_loader)}")

        optimizer.zero_grad()

        spectrograms_tensor = batch_data['audio'].to(device)
        labels = batch_data['player_lvl'].to(device)

        lista_outputs_cls = []
        lista_outputs_reg = []

        labels_long = labels.long()
        labels_float = labels.float()

        # Processando cada clipe
        for i in range(nclips):
            clip_tensor = spectrograms_tensor[:, i, :, :, :]
            logits_cls_clip, output_reg_clip = model(clip_tensor)
            lista_outputs_cls.append(logits_cls_clip)
            lista_outputs_reg.append(output_reg_clip)

        # Calculando a saída média
        logits_cls = torch.stack(lista_outputs_cls).mean(dim=0)
        output_reg = torch.stack(lista_outputs_reg).mean(dim=0)

        # Calculando a perda
        loss_cls = criterion_cls(logits_cls, labels_long)
        output_reg = output_reg.squeeze()
        l1 = criterion_reg_l1(output_reg, labels_float)
        l2 = criterion_reg_l2(output_reg, labels_float)
        loss_reg = l1 + l2

        # Perda total
        loss = (1.0 * loss_cls) + (0.1 * loss_reg)

        # Backpropagation
        loss.backward()
        optimizer.step()

    # --- Bloco de Salvamento ao Final de Cada Época ---
    print(f"--- Fim da Época {epoch+1}. Salvando checkpoint... ---")

    # Crie uma pasta para os checkpoints no seu Drive
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Defina o caminho completo para o arquivo do checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch+1}.pt')

    # Crie o dicionário com tudo que você quer salvar
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,  # Salva a perda do último lote da época
    }, checkpoint_path)

    print(f"Checkpoint salvo em: {checkpoint_path}")


Checkpoint encontrado. Carregando o modelo pré-treinado de ./checkpoints_batch4/model_epoch_100.pt...
Modelo carregado com sucesso a partir da época 100. Continuando o treinamento...


In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error
import numpy as np

# Garante que o modelo está em modo de avaliação (desliga dropout, etc.)
model.eval()

# Cria o dataset de teste para pegar uma amostra
# (Certifique-se que o modo 'test' corresponde ao seu arquivo .pkl de teste)
try:
    test_dataset = PreprocessedDataset(PROCESSED_TEST_DIR)
    test_loader = DataLoader(test_dataset, batch_size=64, num_workers=4)
    print(f"Dataset de teste carregado. Total de amostras: {len(test_dataset)}")
except Exception as e:
    print(f"Erro ao carregar o dataset de teste: {e}")
    print("Verifique se o arquivo 'annotations_unidist_test.pkl' existe na sua pasta de anotações.")
    # Se der erro aqui, pare a execução da célula
    raise

# Armazenar as previsões e os rótulos verdadeiros
all_predictions = []
all_true_labels = []
# Loop para avaliar todo o dataset de teste
for batch_data in test_loader:
    # Prepara os dados para o modelo
    input_tensor = batch_data['audio'].to(device)
    true_labels_batch = batch_data['player_lvl']
    # input_tensor = batch_data['audio'].unsqueeze(0).to(device)
    # true_label = batch_data['player_lvl']

    # Bloco de inferência sem cálculo de gradientes para economizar memória e ser mais rápido
    with torch.no_grad():

        # --- Lógica de inferência para múltiplos clipes (mesma do treino) ---
        clip_outputs_cls = []
        n_clips_from_tensor = input_tensor.shape[1] # Pega o nclips do próprio tensor

        for i in range(n_clips_from_tensor):
            # Pega o i-ésimo clipe
            clip_tensor = input_tensor[:, i, :, :, :]

            # Passa o clipe pelo modelo
            logits_cls_clip, _ = model(clip_tensor)

            # Guarda a saída de classificação
            clip_outputs_cls.append(logits_cls_clip)

        # Agrega as saídas dos clipes tirando a média
        final_logits = torch.stack(clip_outputs_cls).mean(dim=0)
        # --- Fim da lógica de inferência ---

        # Converte os logits em probabilidades
        probabilities = torch.softmax(final_logits, dim=1)

        # Pega a previsão com a maior probabilidade
        # predicted_index = torch.argmax(probabilities, dim=1).item()
        predicted_index = torch.argmax(probabilities, dim=1)


    # Armazena as previsões e os rótulos verdadeiros
    all_predictions.extend(predicted_index.cpu().numpy())
    all_true_labels.extend(true_labels_batch.cpu().numpy())

# --- Cálculo das Métricas ---
accuracy = accuracy_score(all_true_labels, all_predictions)
precision = precision_score(all_true_labels, all_predictions, average='weighted', zero_division=1)
recall = recall_score(all_true_labels, all_predictions, average='weighted', zero_division=1)
f1 = f1_score(all_true_labels, all_predictions, average='weighted', zero_division=1)
mae = mean_absolute_error(all_true_labels, all_predictions)

# Exibe as métricas
print("="*40)
print(f"--- RESULTADOS DA AVALIAÇÃO NO CONJUNTO DE TESTE ---")
print("="*40)
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Precision (weighted): {precision*100:.2f}%")
print(f"Recall (weighted): {recall*100:.2f}%")
print(f"F1 Score (weighted): {f1*100:.2f}%")
print(f"Mean Average Error (MAE): {mae:.2f}")
print("="*40)

Dataset encontrado. Número de amostras: 476
Dataset de teste carregado. Total de amostras: 476
1
2
3
4
5
6
7
8
--- RESULTADOS DA AVALIAÇÃO NO CONJUNTO DE TESTE ---
Accuracy: 60.50%
Precision (weighted): 79.26%
Recall (weighted): 60.50%
F1 Score (weighted): 62.22%
Mean Average Error (MAE): 1.14
